# Class 1. Hello Quantum World

EVA: Quantum Machine Learning | ZHAW | Pavel Sulimov

Goals of this practice session:

1. Set up the quantum computing environment (Qiskit, PennyLane).
2. Work with qubits, superposition, and the Bloch sphere.
3. Run first quantum circuits and interpret measurement results.

## Prerequisites

```
pip install "qiskit>=2.0,<3" "qiskit-aer>=0.15" "pennylane>=0.40" matplotlib numpy
```

Tested with Python 3.11, Qiskit 2.x, PennyLane 0.40+.

---
## Part 1: Math tasks

### M1.1. Probabilities and normalization

Given $|\psi\rangle = \frac{1}{\sqrt{3}}|0\rangle + \sqrt{\frac{2}{3}}\,e^{i\pi/4}|1\rangle$:

1. Compute $P(0)$ and $P(1)$.
2. Verify that $P(0) + P(1) = 1$ (normalization).
3. Does the phase factor $e^{i\pi/4}$ affect the probabilities?

In [ ]:
import numpy as np

alpha = 1 / np.sqrt(3)
beta = np.sqrt(2 / 3) * np.exp(1j * np.pi / 4)

P_0 = np.abs(alpha)**2
P_1 = np.abs(beta)**2

print(f"|psi> = ({alpha:.4f})|0> + ({beta:.4f})|1>")
print(f"P(0) = |alpha|^2 = {P_0:.4f}")
print(f"P(1) = |beta|^2  = {P_1:.4f}")
print(f"P(0) + P(1) = {P_0 + P_1:.4f}  (normalized)")
print(f"\nThe phase e^(i*pi/4) does not affect |beta|^2: "
      f"|sqrt(2/3) * e^(i*pi/4)|^2 = 2/3 = {P_1:.4f}")

### M1.2. Plus and minus states

Write $|+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}$ and $|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}$ in column vector form.

Verify that they are:
1. Normalized: $\langle +|+\rangle = 1$, $\langle -|-\rangle = 1$.
2. Orthogonal: $\langle +|-\rangle = 0$.

In [ ]:
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

ket_plus = (ket_0 + ket_1) / np.sqrt(2)
ket_minus = (ket_0 - ket_1) / np.sqrt(2)

print(f"|+> = {ket_plus}")
print(f"|-> = {ket_minus}")
print(f"\n<+|+> = {np.dot(ket_plus.conj(), ket_plus):.4f}  (normalized)")
print(f"<-|-> = {np.dot(ket_minus.conj(), ket_minus):.4f}  (normalized)")
print(f"<+|-> = {np.dot(ket_plus.conj(), ket_minus):.4f}  (orthogonal)")

### M1.3. Bloch sphere coordinates

A general single-qubit state can be parameterized as $|\psi\rangle = \cos(\theta/2)|0\rangle + e^{i\varphi}\sin(\theta/2)|1\rangle$, which maps to the Bloch vector $(\sin\theta\cos\varphi,\; \sin\theta\sin\varphi,\; \cos\theta)$.

Identify $\theta$ and $\varphi$ for $|0\rangle$, $|1\rangle$, $|+\rangle$, and $|i\rangle = \frac{|0\rangle + i|1\rangle}{\sqrt{2}}$.

In [ ]:
# M1.3 Solution
states = {
    "|0⟩": {"theta": 0, "phi": 0},
    "|1⟩": {"theta": np.pi, "phi": 0},
    "|+⟩": {"theta": np.pi/2, "phi": 0},
    "|−⟩": {"theta": np.pi/2, "phi": np.pi},
    "|i⟩": {"theta": np.pi/2, "phi": np.pi/2},
    "|−i⟩": {"theta": np.pi/2, "phi": 3*np.pi/2},
}

print("State     θ          φ          Bloch (x, y, z)")
print("-" * 55)
for name, data in states.items():
    theta, phi = data["theta"], data["phi"]
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    print(f"{name:5s}  {theta:8.4f}  {phi:8.4f}   ({x:5.2f}, {y:5.2f}, {z:5.2f})")

---
## Part 2: Programming tasks

### P1.1. First Qiskit circuit

In the lecture we saw the Hadamard gate applied to $|0\rangle$, yielding a 50/50 split. Here, build a circuit that creates an *unequal* superposition using a rotation gate $R_y(\pi/3)$.

By default, every qubit in a quantum circuit starts in the state $|0\rangle$. To prepare a different initial state, you would apply gates: for example, `qc.x(0)` flips the qubit to $|1\rangle$ before any other operation.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import plot_histogram

qc = QuantumCircuit(1)
qc.ry(np.pi / 3, 0)
qc.measure_all()

print(qc.draw("text"))
print(f"\nRy(pi/3)|0> = cos(pi/6)|0> + sin(pi/6)|1>")
print(f"Expected: P(0) = cos^2(pi/6) = {np.cos(np.pi/6)**2:.4f}, "
      f"P(1) = sin^2(pi/6) = {np.sin(np.pi/6)**2:.4f}")

In [ ]:
sampler = StatevectorSampler()
job = sampler.run([qc], shots=1024)
result = job.result()

counts = result[0].data.meas.get_counts()
print(f"Counts: {counts}")
print(f"Expected: ~{int(1024*np.cos(np.pi/6)**2)} for |0> and "
      f"~{int(1024*np.sin(np.pi/6)**2)} for |1>")

plot_histogram(counts)

### P1.2. The same rotation in PennyLane

Implement $R_y(\pi/3)|0\rangle$ in PennyLane and compare the result with Qiskit. PennyLane's `default.qubit` device returns exact probabilities by default (no shot noise), so you can verify the analytical answer directly.

In [ ]:
import pennylane as qml
import matplotlib.pyplot as plt

dev = qml.device("default.qubit", wires=1)

theta = np.pi / 3


@qml.qnode(dev)
def ry_circuit():
    """Apply Ry(pi/3) and return outcome probabilities."""
    qml.RY(theta, wires=0)
    return qml.probs(wires=0)


probs = ry_circuit()
print(f"P(|0>) = {probs[0]:.4f},  P(|1>) = {probs[1]:.4f}")
print(f"Analytical: P(|0>) = {np.cos(theta/2)**2:.4f},  "
      f"P(|1>) = {np.sin(theta/2)**2:.4f}")

fig, ax = plt.subplots(figsize=(6, 4))
ax.bar(["|0>", "|1>"], probs, color=["steelblue", "coral"])
ax.set_ylabel("Probability")
ax.set_title(r"$R_y(\pi/3)$ on $|0\rangle$")
ax.set_ylim(0, 1)
plt.tight_layout()
plt.show()

print(qml.draw(ry_circuit)())

### P1.2b. PennyLane with shot noise

On real quantum hardware, you do not get exact probabilities. Instead, you sample individual outcomes. Specifying `shots` in PennyLane simulates this finite-sampling noise.

In [ ]:
dev_shots = qml.device("default.qubit", wires=1, shots=1024)


@qml.qnode(dev_shots)
def ry_circuit_shots():
    qml.RY(theta, wires=0)
    return qml.counts()


counts_pl = ry_circuit_shots()
print(f"Counts (1024 shots): {counts_pl}")
print(f"Expected ratio: ~75% |0>, ~25% |1>. "
      f"Actual results fluctuate due to finite sampling.")

### P1.3. Bloch sphere visualization

In the lecture demo, we visualized the six cardinal states. Here, repeat the exercise with states created by rotation gates at various angles, to build intuition for how gate parameters move the state on the Bloch sphere.

We plot six states: three produced by $R_y(\theta)|0\rangle$ at different angles and three produced by $R_z(\varphi)|+\rangle$ at different phases.

In [ ]:
from qiskit.quantum_info import Statevector


def statevector_to_bloch(sv):
    """Convert a 2-element statevector to Bloch coordinates (x, y, z)."""
    a, b = sv[0], sv[1]
    bx = 2 * np.real(a * np.conj(b))
    by = 2 * np.imag(a * np.conj(b))
    bz = np.abs(a)**2 - np.abs(b)**2
    return [bx, by, bz]


state_defs = {
    "Ry(pi/6)":  [("ry", np.pi / 6, 0)],
    "Ry(pi/3)":  [("ry", np.pi / 3, 0)],
    "Ry(2pi/3)": [("ry", 2 * np.pi / 3, 0)],
    "Rz(0)|+>":  [("h", None, 0)],
    "Rz(pi/2)|+>": [("h", None, 0), ("rz", np.pi / 2, 0)],
    "Rz(pi)|+>":   [("h", None, 0), ("rz", np.pi, 0)],
}

bloch_vectors = {}
for name, gates in state_defs.items():
    qc_tmp = QuantumCircuit(1)
    for gate_name, param, qubit in gates:
        if param is not None:
            getattr(qc_tmp, gate_name)(param, qubit)
        else:
            getattr(qc_tmp, gate_name)(qubit)

    sv = Statevector.from_instruction(qc_tmp)
    bloch_vectors[name] = statevector_to_bloch(sv.data)
    bx, by, bz = bloch_vectors[name]
    print(f"{name:14s}  sv=[{sv.data[0]:.4f}, {sv.data[1]:.4f}]  "
          f"Bloch=({bx:+.2f}, {by:+.2f}, {bz:+.2f})")

In [ ]:
fig, axes = plt.subplots(
    2, 3, figsize=(15, 10), subplot_kw={"projection": "3d"}
)

u = np.linspace(0, 2 * np.pi, 30)
v = np.linspace(0, np.pi, 20)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))

for idx, (name, vec) in enumerate(bloch_vectors.items()):
    ax = axes.flat[idx]
    ax.plot_wireframe(
        x_sphere, y_sphere, z_sphere, alpha=0.1, color="gray"
    )
    for sign in (-1.3, 1.3):
        ax.plot([sign, -sign], [0, 0], [0, 0], "k-", alpha=0.2)
        ax.plot([0, 0], [sign, -sign], [0, 0], "k-", alpha=0.2)
        ax.plot([0, 0], [0, 0], [sign, -sign], "k-", alpha=0.2)

    ax.quiver(
        0, 0, 0, vec[0], vec[1], vec[2],
        color="red", arrow_length_ratio=0.1, linewidth=3,
    )
    ax.scatter([vec[0]], [vec[1]], [vec[2]], color="red", s=100)
    ax.set_title(name, fontsize=14)
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.set_zlim(-1.3, 1.3)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

plt.suptitle(
    "Top row: Ry moves along a meridian (Y-Z plane). "
    "Bottom row: Rz rotates around the equator.",
    fontsize=13,
)
plt.tight_layout()
plt.show()

---
## Bonus

### Bonus 1. Preparing arbitrary initial states

By default, qubits start in $|0\rangle$. What if you want a different starting state? You can apply gates before the main circuit. For example:
- `qc.x(0)` prepares $|1\rangle$
- `qc.h(0)` prepares $|+\rangle$
- `qc.initialize([a, b], 0)` prepares an arbitrary state $a|0\rangle + b|1\rangle$ (Qiskit)

The cell below demonstrates this with Qiskit's `initialize` method and PennyLane's `qml.StatePrep`.

In [ ]:
# Qiskit: initialize an arbitrary state
initial_state = [1 / np.sqrt(3), np.sqrt(2 / 3)]
qc_init = QuantumCircuit(1)
qc_init.initialize(initial_state, 0)
qc_init.measure_all()

counts_init = sampler.run([qc_init], shots=1024).result()[0].data.meas.get_counts()
print(f"Qiskit initialize: {counts_init}")
print(f"Expected: P(0) = {initial_state[0]**2:.3f}, P(1) = {initial_state[1]**2:.3f}")

# PennyLane: StatePrep for the same initial state
dev_init = qml.device("default.qubit", wires=1)


@qml.qnode(dev_init)
def custom_init_circuit():
    qml.StatePrep(np.array(initial_state), wires=0)
    return qml.probs(wires=0)


probs_init = custom_init_circuit()
print(f"\nPennyLane StatePrep: P(|0>) = {probs_init[0]:.4f}, "
      f"P(|1>) = {probs_init[1]:.4f}")

### Bonus 2. Shot noise convergence

A natural question: how many shots do I need for a reliable estimate? The estimation error from finite sampling scales as $\sim 1/\sqrt{N}$ (standard quantum limit). The experiment below verifies this empirically: reducing the error by a factor of 10 requires 100 times more shots.

In [ ]:
theta_test = np.pi / 3  # P(|0>) = cos^2(pi/6) = 0.75
true_prob = np.cos(theta_test / 2) ** 2

shot_counts = [10, 50, 100, 500, 1000, 5000, 10000]
estimated_probs = []
errors = []

for n_shots in shot_counts:
    dev_s = qml.device("default.qubit", wires=1, shots=n_shots)

    @qml.qnode(dev_s)
    def circuit_shots():
        qml.RY(theta_test, wires=0)
        return qml.probs(wires=0)

    p = circuit_shots()
    estimated_probs.append(float(p[0]))
    errors.append(abs(float(p[0]) - true_prob))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.semilogx(shot_counts, estimated_probs, "bo-", ms=8, label="estimated")
ax1.axhline(
    y=true_prob, color="r", ls="--", lw=2,
    label=f"true = {true_prob:.4f}",
)
ax1.set_xlabel("shots")
ax1.set_ylabel(r"$P(|0\rangle)$")
ax1.set_title("Convergence of estimated probability")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.loglog(shot_counts, errors, "ro-", ms=8, label="|error|")
theoretical = [1 / np.sqrt(n) for n in shot_counts]
if errors[0] > 0:
    scale = errors[0] / theoretical[0]
    ax2.loglog(
        shot_counts, [scale * e for e in theoretical],
        "g--", lw=2, label=r"$\sim 1/\sqrt{N}$",
    )
ax2.set_xlabel("shots")
ax2.set_ylabel("absolute error")
ax2.set_title("Error scaling")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"theta = pi/3,  true P(|0>) = {true_prob:.6f}")

---
## Summary

Qubits are described by complex amplitudes $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ with $|\alpha|^2 + |\beta|^2 = 1$. Measurement yields probabilistic outcomes governed by the Born rule: $P(k) = |\langle k|\psi\rangle|^2$. The Bloch sphere provides a geometric picture of single-qubit states.

On the software side, Qiskit and PennyLane offer complementary interfaces to the same underlying linear algebra. Shot noise is an inherent feature of quantum measurement; the estimation error scales as $1/\sqrt{N}$.

Next session: quantum gates, multi-qubit systems, entanglement, and Bell states.